In [1]:
import numpy as np
import pandas as pd
import re
import os
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, KFold

import keras
import tensorflow as tf
import tensorflow_datasets as tfds
# import tensorflow_text
from tensorflow.keras import Sequential, regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, GlobalAveragePooling1D, LSTM, Bidirectional
from tensorflow.python.client import device_lib
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV



# Import and Visualize

### Variables

In [2]:
vocab_size = 5000
oov_token = "<OOV>"
test_size = 0.20
embed_dim=100
max_length = 128
padding_type = "post"
trunction_type = "post"

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

num_folds = 5

### Data Prep

In [3]:
train_data_3 = pd.read_csv("data_v2/Aug_BackTranslation.csv")

from_french = train_data_3[["from_french", "sentiment"]].dropna()
from_french.rename(columns={"from_french": "reviewText"}, inplace=True)

from_italian = train_data_3[["from_italian", "sentiment"]].dropna()
from_italian.rename(columns={"from_italian": "reviewText"}, inplace=True)

from_german = train_data_3[["from_german", "sentiment"]].dropna()
from_german.rename(columns={"from_german": "reviewText"}, inplace=True)

from_chinese = train_data_3[["from_chinese", "sentiment"]].dropna()
from_chinese.rename(columns={"from_chinese": "reviewText"}, inplace=True)

train_data_3 = pd.concat([from_french, from_italian, from_german, from_chinese], ignore_index=True)

val_data_3 = pd.read_csv("data_v2/Aug_Val_BackTranslation.csv")

from_french = val_data_3[["from_french", "sentiment"]].dropna()
from_french.rename(columns={"from_french": "reviewText"}, inplace=True)

from_italian = val_data_3[["from_italian", "sentiment"]].dropna()
from_italian.rename(columns={"from_italian": "reviewText"}, inplace=True)

from_german = val_data_3[["from_german", "sentiment"]].dropna()
from_german.rename(columns={"from_german": "reviewText"}, inplace=True)

from_chinese = val_data_3[["from_chinese", "sentiment"]].dropna()
from_chinese.rename(columns={"from_chinese": "reviewText"}, inplace=True)

val_data_3 = pd.concat([from_french, from_italian, from_german, from_chinese], ignore_index=True)

train_data_1 = pd.read_csv("data_v2/train_data.csv", usecols=["reviewText", "sentiment"])
train_data_2 = pd.read_csv("data_v2/Aug_RandomInsertion.csv", usecols=["reviewText", "sentiment"])
train_data = pd.concat([train_data_1, train_data_2, train_data_3], ignore_index=True)

val_data_1 = pd.read_csv("data_v2/validation_data.csv", usecols=["reviewText", "sentiment"])
val_data_2 = pd.read_csv("data_v2/Aug_Val_RandomInsertion.csv", usecols=["reviewText", "sentiment"])
val_data = pd.concat([val_data_1, val_data_2, val_data_3], ignore_index=True)


test_data = pd.read_csv("data_v2/test_data.csv", usecols=["reviewText", "sentiment"])

In [4]:
train_data = pd.concat([train_data, val_data], ignore_index=True)
train_data.dropna(inplace=True)
train_data = train_data.sample(frac=1)
train_data.reset_index(inplace=True)

In [5]:
X_train = train_data["reviewText"].to_list()
y_train = train_data["sentiment"].tolist()

X_test = test_data["reviewText"].to_list()
y_test = test_data["sentiment"].tolist()



### Tokenizing

In [6]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

### Sequences

In [7]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

### Padding

In [8]:
X_train_padded = pad_sequences(X_train_sequences, 
                               maxlen = max_length, 
                               padding = padding_type, truncating = 
                               trunction_type)
X_test_padded = pad_sequences(X_test_sequences,
                             maxlen = max_length,
                             padding = padding_type,
                             truncating = trunction_type)

## Word Embeddings
### Getting embedding indexes from http://nlp.stanford.edu/data/glove.6B.zip
### Using glove.6B.100d.txt included in this notebook

In [9]:
embeddings_index = {}
f = open('glove.6B/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


### Embedding training set, if a word is not in the stanford embedding index it will be represented as a 0

In [10]:
embedding_matrix = np.zeros((len(word_index) + 1, embed_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Create Embedding Layer

In [14]:
embedding_layer = Embedding(input_dim = len(word_index) + 1,
                           output_dim = embed_dim,
                           weights = [embedding_matrix],
                           input_length = max_length,
                           trainable = False)

### KFold creation

In [15]:
kfold = KFold(n_splits=num_folds, shuffle=True)

## Create Model, compile, train using KFold

In [16]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [43]:
# fold_num = 1
# acc_per_fold = []
# loss_per_fold = []
# for train, test in kfold.split(X_train_padded, y_train):
#     # Model Creation
#     model = Sequential([
#         embedding_layer,
#         Conv1D(128, 5, activation='relu'),
#         Dropout(0.5),
#         Bidirectional(LSTM(32)),
#         Dropout(0.5),
#         Dense(16, kernel_regularizer=regularizers.l2(0.001),activation='relu'),
#         Dense(1, activation='sigmoid')
#     ])
#     # Model Compile
#     model.compile(loss = 'binary_crossentropy', 
#               optimizer = optimizer,
#               metrics = ['accuracy'])
    
#     # Generate a print
#     print('----------------------------------------------------------------------------------------')
#     print(f'Training for fold {fold_num} ...')
    
#     # Fit data to model
#     history = model.fit(X_train_padded[train,], 
#                         y_train[train], 
#                         epochs = 5, 
#                         validation_data = (X_train_padded[test,], y_train[test]))
    
#     # Generalization metrics
#     scores = model.evaluate(X_test_padded, y_test, verbose=0)
#     print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]};\
#           {model.metrics_names[1]} of {scores[1]*100}%')
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])
    
#     fold_num = fold_num + 1

In [44]:
# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

In [45]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validate'], loc = 'upper left')
# plt.show()

In [46]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validate'], loc = 'upper left')
# plt.show()

# This Section I will be testing Hyperparameters Optimization

In [17]:
def model_to_optimize(num_filters, kernel_size):
    model = Sequential([
        embedding_layer,
        Conv1D(num_filters, kernel_size, activation='relu'),
        Dropout(0.5),
        Bidirectional(LSTM(16)),
        Dropout(0.5),
        Dense(16, kernel_regularizer=regularizers.l2(0.001),activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

### Parameters

In [21]:
params = {
    "batch_size": [16, 32],
    "optimizer": ["sgd", "adam"],
    "optimizer__learning_rate": [0.001],
    "model__kernel_size":[3, 5, 7],
    "model__num_filters":[32, 64, 128],
}

### Keras Classifier

In [22]:
model = KerasClassifier(model = model_to_optimize,
                        loss = 'binary_crossentropy',
                        epochs = 10,
                        verbose = True)

### Grid Search

In [23]:
search = GridSearchCV(model, params, cv = 5, verbose=2, scoring='f1')
search.fit(X_train_padded, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Epoch 1/10


2022-12-19 23:12:17.714521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:12:18.041945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:12:18.052483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:12:18.515757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:12:18.530875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


324/324 [==============================] - 32s 93ms/step - loss: 0.6824 - accuracy: 0.5995
Epoch 2/10
324/324 [==============================] - 32s 98ms/step - loss: 0.6475 - accuracy: 0.6544
Epoch 3/10
324/324 [==============================] - 31s 96ms/step - loss: 0.6349 - accuracy: 0.6662
Epoch 4/10
324/324 [==============================] - 31s 96ms/step - loss: 0.6147 - accuracy: 0.6902
Epoch 5/10
324/324 [==============================] - 31s 95ms/step - loss: 0.6077 - accuracy: 0.6902
Epoch 6/10
324/324 [==============================] - 31s 94ms/step - loss: 0.6054 - accuracy: 0.6888
Epoch 7/10
324/324 [==============================] - 28s 86ms/step - loss: 0.6029 - accuracy: 0.6954
Epoch 8/10
324/324 [==============================] - 27s 85ms/step - loss: 0.6030 - accuracy: 0.6954
Epoch 9/10
324/324 [==============================] - 28s 85ms/step - loss: 0.6048 - accuracy: 0.6942
Epoch 10/10
324/324 [==============================] - 26s 81ms/step - loss: 0.5952 - accurac

2022-12-19 23:17:14.014407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:17:14.086599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-19 23:17:14.093355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


81/81 [==============================] - 3s 36ms/step
[CV] END batch_size=16, model__kernel_size=3, model__num_filters=32, optimizer=sgd, optimizer__learning_rate=0.001; total time= 5.0min
Epoch 1/10


...

2022-12-20 09:54:14.983476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:15.228206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:15.234966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


41/41 [==============================] - 5s 80ms/step
[CV] END batch_size=32, model__kernel_size=7, model__num_filters=128, optimizer=adam, optimizer__learning_rate=0.001; total time= 2.7min
Epoch 1/10


2022-12-20 09:54:20.342045: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:21.016205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:21.025427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:23.561010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 09:54:23.573912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


405/405 [==============================] - 56s 123ms/step - loss: 0.6691 - accuracy: 0.6339
Epoch 2/10
405/405 [==============================] - 35s 85ms/step - loss: 0.6350 - accuracy: 0.6732
Epoch 3/10
405/405 [==============================] - 34s 85ms/step - loss: 0.6188 - accuracy: 0.6882
Epoch 4/10
405/405 [==============================] - 34s 85ms/step - loss: 0.6120 - accuracy: 0.6892
Epoch 5/10
405/405 [==============================] - 35s 86ms/step - loss: 0.6088 - accuracy: 0.6908
Epoch 6/10
405/405 [==============================] - 35s 85ms/step - loss: 0.6012 - accuracy: 0.6905
Epoch 7/10
405/405 [==============================] - 35s 86ms/step - loss: 0.5956 - accuracy: 0.6957
Epoch 8/10
405/405 [==============================] - 35s 86ms/step - loss: 0.5945 - accuracy: 0.6933
Epoch 9/10
405/405 [==============================] - 34s 85ms/step - loss: 0.5907 - accuracy: 0.6998
Epoch 10/10
405/405 [==============================] - 35s 86ms/step - loss: 0.5871 - accura

GridSearchCV(cv=5,
             estimator=KerasClassifier(epochs=10, loss='binary_crossentropy', model=<function model_to_optimize at 0x2b4915a60>, verbose=True),
             param_grid={'batch_size': [16, 32],
                         'model__kernel_size': [3, 5, 7],
                         'model__num_filters': [32, 64, 128],
                         'optimizer': ['sgd', 'adam'],
                         'optimizer__learning_rate': [0.001]},
             scoring='f1', verbose=2)

In [25]:
test_accuracy = search.score(X_test_padded, y_test)

2022-12-20 10:49:17.361516: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 10:49:17.621505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-20 10:49:17.650972: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


28/28 [==============================] - 5s 108ms/step


In [26]:
test_accuracy

0.7385103011093502

In [24]:
search.best_params_

{'batch_size': 16,
 'model__kernel_size': 3,
 'model__num_filters': 64,
 'optimizer': 'sgd',
 'optimizer__learning_rate': 0.001}

In [27]:
pd.DataFrame(search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_model__kernel_size,param_model__num_filters,param_optimizer,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,268.056450,14.777681,3.454752,0.104639,16,3,32,sgd,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.734925,0.750460,0.713315,0.754186,0.758752,0.742328,0.016571,9
1,265.535131,9.007239,3.613888,0.245834,16,3,32,adam,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.722298,0.749846,0.744379,0.727398,0.733648,0.735514,0.010276,25
2,273.316967,2.593202,3.543244,0.068934,16,3,64,sgd,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.741722,0.753836,0.739266,0.747297,0.760025,0.748429,0.007667,1
3,273.997022,2.945594,3.659825,0.135806,16,3,64,adam,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.739394,0.747922,0.723549,0.730689,0.745672,0.737445,0.009165,21
4,269.928200,3.564270,3.664378,0.079063,16,3,128,sgd,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.722598,0.743910,0.744472,0.706147,0.748052,0.733036,0.016165,28
5,270.606514,4.537744,3.719197,0.061479,16,3,128,adam,0.001,"{'batch_size': 16, 'model__kernel_size': 3, 'm...",0.734405,0.749073,0.724932,0.756238,0.755414,0.744012,0.012340,4
6,273.721141,4.037149,3.671616,0.061508,16,5,32,sgd,0.001,"{'batch_size': 16, 'model__kernel_size': 5, 'm...",0.731932,0.743573,0.740923,0.740360,0.746926,0.740743,0.004981,12
7,274.440306,4.906639,3.770423,0.103322,16,5,32,adam,0.001,"{'batch_size': 16, 'model__kernel_size': 5, 'm...",0.742379,0.750614,0.733221,0.742347,0.765142,0.746741,0.010721,2
8,270.358288,4.942006,3.914478,0.076849,16,5,64,sgd,0.001,"{'batch_size': 16, 'model__kernel_size': 5, 'm...",0.731334,0.751737,0.745926,0.724719,0.758014,0.742346,0.012478,8
9,274.291005,3.124579,3.927762,0.132488,16,5,64,adam,0.001,"{'batch_size': 16, 'model__kernel_size': 5, 'm...",0.738838,0.750161,0.731169,0.744488,0.730847,0.739100,0.007516,17
